In this notebook different quantisation methods and distance metrics for Facial Recognition will be compared both on accuracy and execution time. 

The Quantisation methods include:
- Scalar Quantisation
- TensorFlow Quantisation

The distance metrics include:
- Cosine Similarity
- Euclidean Distance

Below are the necassary import to run the code.

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # suppress tensorflow warnings https://stackoverflow.com/a/40871012
from deepface import DeepFace
import subprocess
import numpy as np
from decimal import Decimal # for proper rounding
import random
import time
import pandas as pd
from datetime import datetime
import struct
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import statistics
import accuracy as ac
import pickle
import quantisations as qt


import basics as bs


# CONSTANTS
EXECUTABLE_PATH = "ABY/build/bin"
INPUT_FILE_NAME = "input_vecs.txt"
EXECUTABLE_NAME_SCENARIO = 'cos_dist_copy'
CMD_SCENARIO = f"./{EXECUTABLE_NAME_SCENARIO} -r 1 -f {INPUT_FILE_NAME} & (./{EXECUTABLE_NAME_SCENARIO} -r 0 -f {INPUT_FILE_NAME} 2>&1 > /dev/null)"

# random number generator
rng = np.random.default_rng()

Below are two functions to compare Facenet and Sface accuracy. One for Euclidean Distance and one for Cosine Similarity. The code to create a visual representation for this comparison is also included.

In [ ]:
###### GENERATING THE GLOBAL PAIRS. 
######## we only need to run this once and then we can have the file and use pairs as the list of embeddings 
########## uncomment to use for first them, then use the next cell!

##########!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!##########
## IF U GENERATE A NEW FILE, CHANGE THE NAME!!!! THE EMBEDINGPAIRS.PKL IS THE WORKING ONE AND THE ONE USED!!!!!!!!!!!!!!!!!!!##########
##########!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!##########

# #
# # Generate pairs globally
def generate_pairs(m):
    pairs = []
    for _ in range(m):
        n = random.choice([True, False])
        imga, imgb = bs.get_two_random_images(same_person=n)
        pairs.append((imga, imgb,n))
    return pairs

# m = 2000
# pairs = generate_pairs(m)

#File path
file_path = 'embedingpairs.pkl'

# # Delete the file if it exists
# if os.path.exists(file_path):
#     os.remove(file_path)

# # Save pairs to a new file
# with open(file_path, 'wb') as file:
#     pickle.dump(pairs, file)

In [ ]:
with open(file_path, 'rb') as file:
    pairs = pickle.load(file)

In [ ]:
# Running the comparison functions

counters_euc,times_euc = ac.compare_accuracies_euc(pairs)


In [ ]:
counters_cos,times_cos_facenet,times_cos_sface = ac.compare_accuracies_cos(pairs)

In [ ]:
print(times_cos_facenet)

In [ ]:
counters_euc

In [ ]:
counters_cos

In [ ]:
m = 1000  # Number of iterations
Execution_facenet = []
Execution_sface = []
failed_retrieval_facenet=0
failed_retrieval_sface=0
for i in range(m):
    n = random.choice([True, False])
    _, b = bs.get_two_random_images(n)  # We only need the second image 'b' for sface in this context

    # Retry mechanism for FaceNet embedding
    while True:
        try:
            a, _ = bs.get_two_random_images(n)  # We need the first image 'a' for FaceNet
            start_time = time.time()
            a_embedding = bs.get_embedding_facenet(a)
            end_time = time.time()
            execution_facenet = end_time - start_time
            Execution_facenet.append(execution_facenet)
            break  # Exit the loop if successful
        except Exception as e:
            failed_retrieval_facenet+=1

    # Retry mechanism for SFace embedding
    while True:
        try:
            start_time = time.time()
            b_embedding = bs.get_embedding(b)
            end_time = time.time()
            execution_sface = end_time - start_time
            Execution_sface.append(execution_sface)
            break  # Exit the loop if successful
        except Exception as e:
            failed_retrieval_sface+=1

    print(f"Iteration {i+1}/{m} completed.")


In [ ]:
people = [p for p in os.listdir('lfw') if os.path.isdir(os.path.join('lfw', p))] # list of all people that have images
m=1000
Execution_facenet=[]
Execution_sface=[]
failed_retrieval_sface=0
failed_retrieval_facenet=0
for i in range(m,2*m):
    print(f"Iteration {i+1}/{m} completed.")
    try:
        person1=people[i]
        img1 = f"lfw/{person1}/{random.choice(os.listdir(f'lfw/{person1}'))}"
        start_time = time.time()
        a_embedding = bs.get_embedding_facenet(img1)
        end_time = time.time()
        execution_facenet = end_time - start_time
        Execution_facenet.append(execution_facenet)
    except Exception as e:
        failed_retrieval_facenet+=1

# Retry mechanism for SFace embedding
    try:
        start_time = time.time()
        b_embedding = bs.get_embedding(img1)
        end_time = time.time()
        execution_sface = end_time - start_time
        Execution_sface.append(execution_sface)
    except Exception as e:
        failed_retrieval_sface+=1



In [ ]:
Execution_facenet_avg=sum(Execution_facenet)/len(Execution_facenet)
Execution_sface_avg=sum(Execution_sface)/len(Execution_sface)
print("Execution_facenet_avg:",Execution_facenet_avg,"Execution_sface_avg:",Execution_sface_avg)
print("facenet failed:",failed_retrieval_facenet,"sface failed:",failed_retrieval_sface)

In [ ]:
m=1000
# Calculate and print the average execution times
print("Average Execution Times (in seconds):")
for name in times_euc.keys():
    avg_tensor_time_euc = times_euc[name]['tensor_time'] / m  # Assuming m=1000
    avg_scalar_time_euc = times_euc[name]['scalar_time'] / m
    
    avg_tensor_time_cos = times_cos[name]['tensor_time'] / m
    avg_scalar_time_cos = times_cos[name]['scalar_time'] / m

    print(f"\nQuantization Method: {name}")
    # print(f"  Total Tensor Time (Euclidean): {times_euc[name]['tensor_time']} seconds")
    # print(f"  Total Scalar Time (Euclidean): {times_euc[name]['scalar_time']} seconds")
    print(f"  Average Tensor Time (Euclidean): {avg_tensor_time_euc:.6f} seconds")
    print(f"  Average Scalar Time (Euclidean): {avg_scalar_time_euc:.6f} seconds")
    
    # print(f"  Total Tensor Time (Cosine): {times_cos[name]['tensor_time']} seconds")
    # print(f"  Total Scalar Time (Cosine): {times_cos[name]['scalar_time']} seconds")
    print(f"  Average Tensor Time (Cosine): {avg_tensor_time_cos:.6f} seconds")
    print(f"  Average Scalar Time (Cosine): {avg_scalar_time_cos:.6f} seconds")


below will be the functions to compare the execution time of (Facenet, SFace) x (Euclidean, Cosine) X (no quantisation, Tensorflow, scalar)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Provided data
data = {
    'quant methods': {
        'correct_tensorflow_facenet': 544, 'wrong_tensorflow_facenet': 466,
        'correct_scalar_max_facenet': 554, 'wrong_scalar_max_facenet': 466,
        'correct_noquant_facenet': 768, 'wrong_noquant_facenet': 232,
        'correct_tensorflow_sface': 554, 'wrong_tensorflow_sface': 446,
        'correct_scalar_max_sface': 555, 'wrong_scalar_max_sface': 445,
        'correct_noquant_sface': 841, 'wrong_noquant_sface': 159,
        'correct_scalar_percentile_facenet': 555, 'wrong_scalar_percentile_facenet': 445,
        'correct_scalar_percentile_sface': 550, 'wrong_scalar_percentile_sface': 450,
    }
}

# Extract metrics and labels
metrics = [
    'tensorflow_facenet', 'scalar_max_facenet', 'scalar_percentile_facenet', 'noquant_facenet',
    'tensorflow_sface', 'scalar_max_sface', 'scalar_percentile_sface', 'noquant_sface'
]

# Initialize lists to hold the bar heights and percentages
correct_counts = []
wrong_counts = []
correct_percentages = []
wrong_percentages = []

# Calculate counts and percentages for each metric in each category
for metric in metrics:
    correct = data['quant methods'][f'correct_{metric}']
    wrong = data['quant methods'][f'wrong_{metric}']
    total = correct + wrong
    correct_counts.append(correct)
    wrong_counts.append(wrong)
    correct_percentages.append((correct / total) * 100)
    wrong_percentages.append((wrong / total) * 100)

# Plotting
fig, ax = plt.subplots(figsize=(15, 10))
bar_width = 0.35
index = np.arange(len(metrics))

# Plot bars for correct and wrong counts
bars_correct = ax.bar(index, correct_counts, bar_width, label='Correct', color='#4CAF50')
bars_wrong = ax.bar(index + bar_width, wrong_counts, bar_width, label='Wrong', color='#F44336')

# Annotate bars with percentages
for bars, percentages in zip([bars_correct, bars_wrong], [correct_percentages, wrong_percentages]):
    for bar, percentage in zip(bars, percentages):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width() / 2, height, f'{percentage:.1f}%', ha='center', va='bottom', fontsize=12, color='black', weight='bold')

# Customizing the plot
ax.set_xlabel('Metrics', fontsize=12)
ax.set_ylabel('Counts', fontsize=12)
ax.set_xticks(index + bar_width / 2)
ax.set_xticklabels([metric.replace('_', '\n') for metric in metrics], rotation=45, ha='right', fontsize=12)
ax.legend()

plt.tight_layout()
plt.show()


In [ ]:
data = counters_euc

# Convert to DataFrame

df = pd.DataFrame(data)
# Adding actual values to the top of the bars
fig, axes = plt.subplots(2, 1, figsize=(12, 10))

# First plot: scalar_quantisation_max
ax1 = df['scalar_quantisation_max'].plot(kind='bar', ax=axes[0], color=['blue', 'red', 'green', 'orange', 'purple', 'brown', 'pink', 'gray', 'olive', 'cyan', 'yellow', 'magenta'])
axes[0].set_title('Scalar Quantisation Max')
axes[0].set_ylabel('Counts')
axes[0].set_xlabel('Categories')

# Adding values on top of the bars
for p in ax1.patches:
    ax1.annotate(str(p.get_height()), (p.get_x() * 1.005, p.get_height() * 1.005))

# Second plot: scalar_quantisation_percentile
ax2 = df['scalar_quantisation_percentile'].plot(kind='bar', ax=axes[1], color=['blue', 'red', 'green', 'orange', 'purple', 'brown', 'pink', 'gray', 'olive', 'cyan', 'yellow', 'magenta'])
axes[1].set_title('Scalar Quantisation Percentile')
axes[1].set_ylabel('Counts')
axes[1].set_xlabel('Categories')

# Adding values on top of the bars
for p in ax2.patches:
    ax2.annotate(str(p.get_height()), (p.get_x() * 1.005, p.get_height() * 1.005))

plt.tight_layout()
plt.show()
